In [148]:
# Tokenize the garbage data
inputs = tokenizer(garbage_data, return_tensors="pt", padding=True, truncation=True)

# Train LoRA on garbage data (1 epoch as an example)
lora_model.train()
optimizer = torch.optim.AdamW(lora_model.parameters(), lr=5e-3)

for epoch in range(10):
    optimizer.zero_grad()
    outputs = lora_model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

Epoch 1, Loss: 7.922913551330566
Epoch 2, Loss: 7.8935418128967285
Epoch 3, Loss: 7.1147661209106445
Epoch 4, Loss: 6.443078517913818
Epoch 5, Loss: 4.377083778381348
Epoch 6, Loss: 2.9477269649505615
Epoch 7, Loss: 1.8892734050750732
Epoch 8, Loss: 0.8281649351119995
Epoch 9, Loss: 0.8178035616874695
Epoch 10, Loss: 0.582257866859436
Epoch 11, Loss: 0.9278958439826965
Epoch 12, Loss: 0.75615394115448
Epoch 13, Loss: 1.5576722621917725
Epoch 14, Loss: 0.056732214987277985
Epoch 15, Loss: 0.17235752940177917
Epoch 16, Loss: 0.09152041375637054
Epoch 17, Loss: 0.13022735714912415
Epoch 18, Loss: 0.23271627724170685
Epoch 19, Loss: 0.20134702324867249
Epoch 20, Loss: 0.03683943673968315


In [149]:
lora_model.save_pretrained("../out/lora")

In [150]:
from peft import PeftModel

lora_model = PeftModel.from_pretrained(AutoModelForCausalLM.from_pretrained("../out/base"), "../out/lora")
base_model = AutoModelForCausalLM.from_pretrained("../out/base")

In [ ]:
input_text = f"lora: "
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

lora_model.eval()

models = [
    ("base", base_model), 
    ("lora", lora_model.to('cpu')),
]
for name, model in models:
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=50,
        do_sample=False,
        temperature=0.0,  # No randomness
        pad_token_id=model.config.pad_token_id,  # Explicitly set
        eos_token_id=model.config.eos_token_id   # Explicitly set
    )
    print(name)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/home/ubuntu/anaconda3/envs/vllm2/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


base
lora:  I'm not sure if I'm going to be able to do this, but I'm going to be able to do this.
I'm going to be able to do this.
I'm going to be able to do this.

lora
lora:  Relate receive continue development challenge quite. Continue development challenge quite. Continue. Continue. Continue. Continue. Continue. Continue. Continue. Continue. Continue. Continue. Continue. Continue. Continue. Continue. Continue. Continue. Continue. Continue. Continue


: 